In [1]:
import typing

import pandas as pd

import config
import src
import requests
import tqdm
import json
import numpy as np

from sklearn.metrics import cohen_kappa_score, classification_report
import krippendorff
import yaml

import os
from dotenv import load_dotenv
load_dotenv("sjoerdAzure.env")  # Load environment variables from .env file
import time


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "c:\Users\sstolwi\Github\llmdiv\Lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "c:\Users\sstolwi\Github\llmdiv\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "c:\Users\sstolwi\Github\llmdiv\Lib\site-packages\ipykernel\kernelapp.py", line 739, in start
    self.io_loop.start()
  File "c:\Users\sstolwi\Git

In [2]:
CFG = config.Config()

In [3]:
dataset: pd.DataFrame = pd.read_csv('data/publicsphere/full_data.csv')
dataset

,StartDate,RecordedDate,IPAddress,Finished,Coder,ID,Mark_ID,Genre,topiccode,Platform,...,dislikeCount_video,likeCount_video,date_difference,commentCount_video,replyCount_comment,topic,subscribers,HATELIST_FOCUSED_DUMMY,Time_comment_year,Time_video_year
0,5/30/2021 13:03:17,5/30/2021 13:04:17,62.194.51.29,1,6,UgyPHwv8G0cDE6-wEgl4AaABAg.8_0ZjJKSJty8_0kXGkAd2U,119,0,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,2017,2017.0
1,10/11/2021 10:34:05,10/11/2021 10:36:46,213.127.109.191,1,6,Ugx2WXq9UdV8mPPjejJ4AaABAg.8yHCKV0Boe58yYRxEQEF45,282,1,2,1,...,195.0,3817.0,743.0,1748.0,NaN,economy,3630000.0,0,2019,2019.0
2,9/9/2021 18:49:48,9/9/2021 18:51:32,213.127.110.0,1,6,1110578710648890000,372,2,4,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,2019,NaN
3,6/6/2021 16:12:46,6/6/2021 16:16:16,213.127.76.145,1,6,UgwUPFScjJ0MCeaP2F54AaABAg.8lvp3fc9Euf8lvvgsUgEgV,769,0,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,2018,2018.0
4,6/13/2021 13:25:49,6/13/2021 13:27:28,213.127.82.232,1,6,UgwWKCWtSJdFvjGHvTp4AaABAg.8kUC5dGrQ2H8kUDRihE2f3,1206,0,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,2018,2018.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3857,8/19/2021 14:50:13,8/19/2021 14:54:28,62.194.51.29,1,6,1152219467579100000,10000695,0,4,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,2019,NaN
3858,8/19/2021 15:10:27,8/19/2021 15:12:21,62.194.51.29,1,6,1085362296472430000,10007008,1,4,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,2019,NaN
3859,10/6/2021 16:08:39,10/6/2021 16:10:42,213.127.113.113,1,6,UghFY3QJ6nmT_ngCoAEC.7-H0Z7--wxd8goqpaPs-bl,20000102,0,3,1,...,2820.0,12475.0,3803.0,4785.0,NaN,east,6740000.0,0,2018,2010.0
3860,10/15/2021 18:30:04,10/15/2021 18:35:40,213.127.109.191,1,6,UgyWabsmmnq3zam4DgZ4AaABAg,20000418,2,3,1,...,118.0,31761.0,1531.0,2206.0,0.0,east,6800000.0,0,2018,2015.0


In [4]:
#load train/test data
trainset: pd.DataFrame = pd.read_csv('data/publicsphere/train.csv')
testset: pd.DataFrame = pd.read_csv('data/publicsphere/test.csv')

In [5]:
trainset.head()

,StartDate,RecordedDate,IPAddress,Finished,Coder,ID,Mark_ID,Genre,topiccode,Platform,...,dislikeCount_video,likeCount_video,date_difference,commentCount_video,replyCount_comment,topic,subscribers,HATELIST_FOCUSED_DUMMY,Time_comment_year,Time_video_year
0,5/30/2021 13:03:17,5/30/2021 13:04:17,62.194.51.29,1,6,UgyPHwv8G0cDE6-wEgl4AaABAg.8_0ZjJKSJty8_0kXGkAd2U,119,0,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,2017,2017.0
1,9/9/2021 18:49:48,9/9/2021 18:51:32,213.127.110.0,1,6,1110578710648890000,372,2,4,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,2019,NaN
2,6/13/2021 13:25:49,6/13/2021 13:27:28,213.127.82.232,1,6,UgwWKCWtSJdFvjGHvTp4AaABAg.8kUC5dGrQ2H8kUDRihE2f3,1206,0,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,2018,2018.0
3,11/30/2021 17:24:18,11/30/2021 17:27:16,213.127.109.191,1,6,Ugw2eTvkZLfH9MDVg1R4AaABAg,1214,1,2,1,...,662.0,9472.0,743.0,2912.0,0.0,economy,3630000.0,1,2019,2018.0
4,6/18/2021 17:59:54,6/18/2021 18:01:59,213.127.126.118,1,6,Ugi7W7rSRtQSQngCoAEC.8SSDZOcrB8y8SSIYSKC0FI,1963,0,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2017,2017.0


In [5]:
#add train/test split variable to the dataset:
dataset.loc[dataset['Mark_ID'].isin(trainset['Mark_ID']), 'split'] = 'train'
dataset.loc[dataset['Mark_ID'].isin(testset['Mark_ID']), 'split'] = 'test'

In [15]:
dataset['split'].value_counts()

split
train    3089
test      773
Name: count, dtype: int64

In [4]:
#set option variables:

#set options to low temperature (0,1):
options_low_str = """
seed: 42
temperature: 0.1
"""

options_low = yaml.safe_load(options_low_str)

MODELsmall: str = 'llama3.1:8b-instruct-q6_K' # options: 'gemma:7b-instruct-q6_K', 'gemma2:27b-instruct-q6_K', 'llama3.1:8b-instruct-q6_K', 'llama3.1:70b-instruct-q6_K', 'mistral:7b-instruct-v0.3-q6_K', 'mistral-large:123b-instruct-2407-q6_K', 'mixtral:8x7b-instruct-v0.1-q6_K', 'mixtral:8x22b-instruct-v0.1-q6_K', 'phi3:14b-medium-128k-instruct-q6_K' or 'qwen2:72b-instruct-q6_K'
MODELlarge: str = 'llama3.1:70b-instruct-q6_K' # options: 'gemma:7b-instruct-q6_K', 'gemma2:27b-instruct-q6_K', 'llama3.1:8b-instruct-q6_K', 'llama3.1:70b-instruct-q6_K', 'mistral:7b-instruct-v0.3-q6_K', 'mistral-large:123b-instruct-2407-q6_K', 'mixtral:8x7b-instruct-v0.1-q6_K', 'mixtral:8x22b-instruct-v0.1-q6_K', 'phi3:14b-medium-128k-instruct-q6_K' or 'qwen2:72b-instruct-q6_K'
MODELgpt4o = "nf-gpt-4o-2024-08-06" # in principe is er nu van elk model een nf (no filter) en een normale versie beschikbaar, de no filter versies zijn alleen voor onderzoekers beschikbaar voor analyze van content die niet door de filter heen zou komen.
MODELgpt4T = "nf-gpt-4-turbo" # Can be gpt-35-turbo, gpt-4-turbo, gpt-4 or Meta-Llama-3-8B-Instruct.

options_zero_str = """
seed: 42
temperature: 0
"""
options_zero = yaml.safe_load(options_zero_str)

temperature_0 : int = 0
SEED: int = 42
MAX10: int = 10
TOPP1: int = 1


options_large_str = """
seed: 42
temperature: 0
num_predict: 2000
"""
options_large = yaml.safe_load(options_large_str)

#load environment variables:
api_key = os.environ.get('sjoerd_key')

#setttings:
api_endpoint = "https://ai-research-proxy.azurewebsites.net/chat/completions"
####### API REQUEST FORMATTING ######
headers = {
    "Content-Type": "application/json",
    "Authorization": "Bearer " + api_key
}
 

In [6]:
#load the dataset:
#note this dataset already contains the output, do not try to overwrite it.
dataset_w_pred_anon = pd.read_parquet('data/publicsphere/publicsphere.cardiff_prompt_classify_anon.parquet')

In [ ]:
#add train/test split variable to the dataset:
dataset_w_pred_anon.loc[dataset_w_pred_anon['Mark_ID'].isin(trainset['Mark_ID']), 'split'] = 'train'
dataset_w_pred_anon.loc[dataset_w_pred_anon['Mark_ID'].isin(testset['Mark_ID']), 'split'] = 'test'

In [ ]:
pubspherepromptsrun1 = ['incivility_namecalling_edit']   #repeat for all prompts
predictions1: typing.Dict[str, np.ndarray] = {
    label: (
        src.PromptClassify
        .from_json(path)
        (dataset["commentText"], model=MODELlarge, options=options_low)
    )
    for label, path in CFG.prompt_classify_files.items() if label in pubspherepromptsrun1
}

In [42]:
#join to the dataset:   
for _, preds in predictions1.items():
    print(preds.value_counts())
    print("-" * 42)
    dataset_w_pred_2 = dataset_w_pred_2.join(preds)
    dataset_w_pred_anon = dataset_w_pred_anon.join(preds)

incivility_namecalling
No     2670
Yes    1189
Name: count, dtype: int64
------------------------------------------
incivility_namecalling_edit
No     2486
Yes    1371
Name: count, dtype: int64
------------------------------------------


In [57]:
#make dummy variables:
dataset_w_pred_anon.loc[:, 'incivility_allcaps_dum'] = dataset_w_pred_anon.loc[:, 'incivility_allcaps'].map({"Yes": 1, "No":0}).fillna(0).astype(int)
dataset_w_pred_anon.loc[:, 'incivility_attackreputation_dum'] = dataset_w_pred_anon.loc[:, 'incivility_attackreputation'].map({"Yes": 1, "No":0}).fillna(0).astype(int)
dataset_w_pred_anon.loc[:, 'incivility_question_intelligence_dum'] = dataset_w_pred_anon.loc[:, 'incivility_question_intelligence'].map({"Yes": 1, "No":0}).fillna(0).astype(int)
dataset_w_pred_anon.loc[:, 'incivility_sarcasm_dum'] = dataset_w_pred_anon.loc[:, 'incivility_sarcasm'].map({"Yes": 1, "No":0}).fillna(0).astype(int)
dataset_w_pred_anon.loc[:, 'incivility_namecalling_dum'] = dataset_w_pred_anon.loc[:, 'incivility_namecalling'].map({"Yes": 1, "No":0}).fillna(0).astype(int)
dataset_w_pred_anon.loc[:, 'incivility_vulgarity_dum'] = dataset_w_pred_anon.loc[:, 'incivility_vulgarity'].map({"Yes": 1, "No":0}).fillna(0).astype(int)


dataset_w_pred_anon.loc[:, 'interactivity_acknowledgement_dum'] = dataset_w_pred_anon.loc[:, 'interactivity_acknowledgement_low'].map({"Yes": 1, "No":0}).fillna(0).astype(int)
dataset_w_pred_anon.loc[:, 'rationality_external_evidence_dum'] = dataset_w_pred_anon.loc[:, 'rationality_external_evidence_low'].map({"Yes": 1, "No":0}).fillna(0).astype(int)
dataset_w_pred_anon.loc[:, 'rationality_topic_relevance_dum'] = dataset_w_pred_anon.loc[:, 'rationality_topic_relevance_low'].map({"Yes": 1, "No":0}).fillna(0).astype(int)
dataset_w_pred_anon.loc[:, 'rationality_background_info_low_dum'] = dataset_w_pred_anon.loc[:, 'rationality_background_info_low'].map({"Yes": 1, "No":0}).fillna(0).astype(int)
dataset_w_pred_anon.loc[:, 'rationality_reasoning_dum'] = dataset_w_pred_anon.loc[:, 'rationality_reasoning_low'].map({"Yes": 1, "No":0}).fillna(0).astype(int)
dataset_w_pred_anon.loc[:, 'intolerance_discrimination_dum'] = dataset_w_pred_anon.loc[:, 'intolerance_discrimination'].map({"Yes": 1, "No":0}).fillna(0).astype(int)
dataset_w_pred_anon.loc[:, 'intolerance_rights_dum'] = dataset_w_pred_anon.loc[:, 'intolerance_rights'].map({"Yes": 1, "No":0}).fillna(0).astype(int)   
dataset_w_pred_anon.loc[:, 'intolerance_violence_dum'] = dataset_w_pred_anon.loc[:, 'intolerance_violence'].map({"Yes": 1, "No":0}).fillna(0).astype(int)
dataset_w_pred_anon.loc[:, 'incivility_namecalling_dum'] = dataset_w_pred_anon.loc[:, 'incivility_namecalling'].map({"Yes": 1, "No":0}).fillna(0).astype(int)

dataset_w_pred_anon.loc[:, 'political_conservative_US_dum'] = dataset_w_pred_anon.loc[:, 'political_ideology_US_low'].map({"conservative": 1}).fillna(0).astype(int)
dataset_w_pred_anon.loc[:, 'political_liberal_US_dum'] = dataset_w_pred_anon.loc[:, 'political_ideology_US_low'].map({"liberal": 1}).fillna(0).astype(int)

dataset_w_pred_anon.loc[:, 'diversity_conservative_dum'] = dataset_w_pred_anon.loc[:, 'diversity_ideologicaldirection'].map({"Left/Liberal/Democratic": 0, "Absent":0, "Neutral":0, "Unclear which direction":0 , "Right/Conservative/Republican":1}).fillna(0).astype(int)
dataset_w_pred_anon.loc[:, 'diversity_liberal_dum'] = dataset_w_pred_anon.loc[:, 'diversity_ideologicaldirection'].map({"Left/Liberal/Democratic": 1, "Absent":0, "Neutral":0, "Unclear which direction":0 , "Right/Conservative/Republican":0}).fillna(0).astype(int)

dataset_w_pred_anon.loc[:, 'interactivity_acknowledgement_updated_dum'] = dataset_w_pred_anon.loc[:, 'interactivity_acknowledgement_updated'].map({"Yes": 1, "No":0}).fillna(0).astype(int)
dataset_w_pred_anon.loc[:, 'rationality_background_info_updated_dum'] = dataset_w_pred_anon.loc[:, 'rationality_background_info_updated'].map({"Yes": 1, "No":0}).fillna(0).astype(int)
dataset_w_pred_anon.loc[:, 'rationality_external_evidence_updated_dum'] = dataset_w_pred_anon.loc[:, 'rationality_external_evidence_updated'].map({"Yes": 1, "No":0}).fillna(0).astype(int)
dataset_w_pred_anon.loc[:, 'incivility_namecalling_updated_dum'] = dataset_w_pred_anon.loc[:, 'incivility_namecalling_updated'].map({"Yes": 1, "No":0}).fillna(0).astype(int)

dataset_w_pred_anon.loc[:, 'incivility_namecalling_edit_dum'] = dataset_w_pred_anon.loc[:, 'incivility_namecalling_edit'].map({"Yes": 1, "No":0}).fillna(0).astype(int)
dataset_w_pred_anon.loc[:, 'rationality_reasoning_edit_dum'] = dataset_w_pred_anon.loc[:, 'rationality_reasoning_edit'].map({"Yes": 1, "No":0}).fillna(0).astype(int)

dataset_w_pred_anon.loc[:, 'rationality_external_evidence_edit_dum'] = dataset_w_pred_anon.loc[:, 'rationality_external_evidence_edit'].map({"Yes": 1, "No":0}).fillna(0).astype(int)
dataset_w_pred_anon.loc[:, 'rationality_background_info_edit_dum'] = dataset_w_pred_anon.loc[:, 'rationality_background_info_edit'].map({"Yes": 1, "No":0}).fillna(0).astype(int)



In [58]:
#create compound-concept variables with the latest version of the prompt:
dataset_w_pred_anon.loc[:, 'rationality_prompt_dum'] = dataset_w_pred_anon.loc[:, ['rationality_reasoning_edit_dum','rationality_background_info_edit_dum','rationality_external_evidence_edit_dum']].max(axis=1)
dataset_w_pred_anon.loc[:, 'incivility_prompt_dum'] = dataset_w_pred_anon.loc[:, ['incivility_namecalling_edit_dum', 'incivility_vulgarity_dum', 'incivility_attackreputation_dum', 
                'incivility_question_intelligence_dum', 'incivility_allcaps_dum', 'incivility_sarcasm_dum',
              'intolerance_rights_dum', 'intolerance_discrimination_dum', 'intolerance_violence_dum']].max(axis=1)


In [14]:
dataset_w_pred_anon.loc[:, ['rationality_prompt_dum', 'rationality_reasoning_dum','rationality_background_info_low_dum','rationality_external_evidence_dum', 'rationality_external_evidence_low','rationality_background_info_low','rationality_reasoning_low' ]].head(10)

,rationality_prompt_dum,rationality_reasoning_dum,rationality_background_info_low_dum,rationality_external_evidence_dum,rationality_external_evidence_low,rationality_background_info_low,rationality_reasoning_low
0,0,0,0,0,No,No,No
1,1,1,0,0,No,No,Yes
2,1,1,1,1,Yes,Yes,Yes
3,1,1,0,0,No,No,Yes
4,1,1,0,0,No,No,Yes
5,0,0,0,0,None,No,No
6,0,0,0,0,None,No,No
7,0,0,0,0,None,No,No
8,1,1,0,0,No,No,Yes
9,1,1,0,0,No,No,Yes


In [59]:
#count NA's:
dataset_w_pred_anon.loc[:, ['interactivity_acknowledgement_low', 'rationality_external_evidence_low','rationality_background_info_low','rationality_reasoning_low', 'political_ideology_US_low', "rationality_topic_relevance_low", 
                            "political_ideology_US_low", 'incivility_namecalling', 'incivility_vulgarity', 'incivility_attackreputation', 
                            'incivility_question_intelligence', 'incivility_allcaps', 'incivility_sarcasm', 'intolerance_rights', 'intolerance_discrimination', 'intolerance_violence', 
                            'diversity_ideologicaldirection', 'diversity_position', 'diversity_disagreement', 'rationality_background_info_updated', 'rationality_external_evidence_updated',
                            'interactivity_acknowledgement_updated', 'incivility_namecalling_updated',
                            'incivility_namecalling_edit','rationality_reasoning_edit', 'rationality_external_evidence_edit', 
                            'rationality_background_info_edit']].isna().sum()

interactivity_acknowledgement_low           2
rationality_external_evidence_low         910
rationality_background_info_low            90
rationality_reasoning_low                 102
political_ideology_US_low                   0
rationality_topic_relevance_low             2
political_ideology_US_low                   0
incivility_namecalling                    156
incivility_namecalling_NA                 174
incivility_vulgarity                       16
incivility_attackreputation                 9
incivility_question_intelligence            3
incivility_allcaps                          5
incivility_sarcasm                          6
intolerance_rights                          3
intolerance_discrimination                  1
intolerance_violence                        3
diversity_ideologicaldirection              5
diversity_position                          3
diversity_disagreement                      4
rationality_background_info_updated       151
rationality_external_evidence_upda

In [ ]:
#Note many missing values...especially in external_evidence.
#but edited prompts fixed the issue, so we can use the edited prompts for the analysis.

In [ ]:
#what if we combine items into one simple prompt:
pubspherepromptsrun21 = ['interactivity_acknowledgement_simple', 'incivility_simple2', 'rationality_simple2']   
predictions21: typing.Dict[str, np.ndarray] = {
    label: (
        src.PromptClassify
        .from_json(path)
        (dataset["commentText"], model=MODELlarge, options=options_low)
    )
    for label, path in CFG.prompt_classify_files.items() if label in pubspherepromptsrun21
}

classifying rationality_simple2: 100%|██████████| 3862/3862 [46:36<00:00,  1.38it/s] 


In [ ]:
#join to the dataset:   
for _, preds in predictions21.items():
    print(preds.value_counts())
    print("-" * 42)
     
    dataset_w_pred_2 = dataset_w_pred_2.join(preds)
    dataset_w_pred_anon = dataset_w_pred_anon.join(preds)

In [ ]:
dataset_w_pred_anon.loc[:, 'incivility_simple2_dum'] = dataset_w_pred_2.loc[:, 'incivility_simple2'].map({"Yes": 1, "No":0}).fillna(0).astype(int)
dataset_w_pred_anon.loc[:, 'rationality_simple2_dum'] = dataset_w_pred_2.loc[:, 'rationality_simple2'].map({"Yes": 1, "No":0}).fillna(0).astype(int)
dataset_w_pred_anon.loc[:, 'interactivity_acknowledgement_simple_dum'] = dataset_w_pred_2.loc[:, 'interactivity_acknowledgement_simple'].map({"Yes": 1, "No":0}).fillna(0).astype(int)


In [ ]:
#evaluete respective performance on the TRAINING set (NOTE the wording of the few shot examples in the prompt is polished by GPT4o, not exactly like codebook):
llm_human_column_pairs = [
    ("RATIONALITY_DUMMY", "rationality_simple2_dum"),
    ("INCIVILITY_DUMMY", "incivility_simple2_dum"),
    ("INTERACTIVITY_DUMMY", "interactivity_acknowledgement_simple_dum"),
    ("LIBERAL_DUMMY", "political_liberal_US_dum"),
    ("CONSERVATIVE_DUMMY", "political_conservative_US_dum"),
    ("HAS_OPINION_DUMMY", "political_opinion_US_dum")
]
print("Inter-coder reliablity: classification_report(human as gold, llm as predicted), cohen_kappa(human, llm):")

for human_col, llm_col in llm_human_column_pairs:
    subset = dataset_w_pred_anon.loc[dataset_w_pred_anon.split == 'train', [human_col, llm_col]].dropna()
    human = subset[human_col].tolist()
    llm = subset[llm_col].tolist()

    # Calculate percent agreement
    agreement = np.sum(np.array(human) == np.array(llm))/len(np.array(human)) * 100

    print("---")
    print(f"{classification_report(human, llm)}")
    print(f"cohen_kappa_score({human_col}): {cohen_kappa_score(human, llm)}")
    print(f"krippendorf({human_col}): {krippendorff.alpha(np.array([human, llm]), level_of_measurement="nominal")}")
    print(f"percent_agreement({human_col}): {agreement:.2f}%")


In [ ]:
#what if we would use the lightweight version of the model:
MODELsmall: str = 'llama3.1:8b-instruct-q6_K' # options: 'gemma:7b-instruct-q6_K', 'gemma2:27b-instruct-q6_K', 'llama3.1:8b-instruct-q6_K', 'llama3.1:70b-instruct-q6_K', 'mistral:7b-instruct-v0.3-q6_K', 'mistral-large:123b-instruct-2407-q6_K', 'mixtral:8x7b-instruct-v0.1-q6_K', 'mixtral:8x22b-instruct-v0.1-q6_K', 'phi3:14b-medium-128k-instruct-q6_K' or 'qwen2:72b-instruct-q6_K'

pubspherepromptsrun25 = ['interactivity_acknowledgement_simple', 'political_ideology_US', 'incivility_simple2', 'rationality_simple2']   
predictions21: typing.Dict[str, np.ndarray] = {
    label: (
        src.PromptClassify
        .from_json(path)
        (dataset["commentText"], model=MODELsmall, options=options_low)
    )
    for label, path in CFG.prompt_classify_files.items() if label in pubspherepromptsrun25
}

classifying rationality_simple2: 100%|██████████| 3862/3862 [16:50<00:00,  3.82it/s]


In [32]:
#join to the dataset:   
for _, preds in predictions21.items():
    print(preds.value_counts())
    print("-" * 42)
     
    dataset_w_pred_2 = dataset_w_pred_2.join(preds, rsuffix='_small')
    dataset_w_pred_anon = dataset_w_pred_anon.join(preds, rsuffix='_small')

interactivity_acknowledgement_simple
No     2040
Yes    1697
Name: count, dtype: int64
------------------------------------------
rationality_simple2
No     3681
Yes     155
Name: count, dtype: int64
------------------------------------------


In [ ]:
dataset_w_pred_anon.loc[:, 'incivility_simple2_small_dum'] = dataset_w_pred_2.loc[:, 'incivility_simple2_small'].map({"Yes": 1, "No":0}).fillna(0).astype(int)
dataset_w_pred_anon.loc[:, 'rationality_simple2_small_dum'] = dataset_w_pred_2.loc[:, 'rationality_simple2_small'].map({"Yes": 1, "No":0}).fillna(0).astype(int)
dataset_w_pred_anon.loc[:, 'interactivity_acknowledgement_simple_small_dum'] = dataset_w_pred_2.loc[:, 'interactivity_acknowledgement_simple_small'].map({"Yes": 1, "No":0}).fillna(0).astype(int)

dataset_w_pred_anon.loc[:, 'political_conservative_US_small_dum'] = dataset_w_pred_anon.loc[:, 'political_ideology_US'].map({"conservative": 1}).fillna(0).astype(int)
dataset_w_pred_anon.loc[:, 'political_liberal_US_small_dum'] = dataset_w_pred_anon.loc[:, 'political_ideology_US'].map({"liberal": 1}).fillna(0).astype(int)
dataset_w_pred_anon.loc[:, 'political_opinion_US_small_dum'] = dataset_w_pred_anon.loc[:, 'political_ideology_US'].map({"liberal": 1, 'conservative': 1}).fillna(0).astype(int)


In [ ]:
#evaluate performance of small model on the TRAINING set:
llm_human_column_pairs = [
    ("RATIONALITY_DUMMY", "rationality_simple2_small_dum"),
    ("INCIVILITY_DUMMY", "incivility_simple2_small_dum"),
    ("INTERACTIVITY_DUMMY", "interactivity_acknowledgement_simple_small_dum"),
    ("LIBERAL_DUMMY", "political_liberal_US_small_dum"),
    ("CONSERVATIVE_DUMMY", "political_conservative_US_small_dum")
]
print("Inter-coder reliablity: classification_report(human as gold, llm as predicted), cohen_kappa(human, llm):")

for human_col, llm_col in llm_human_column_pairs:
    subset = dataset_w_pred_anon.loc[dataset_w_pred_anon.split == 'train', [human_col, llm_col]].dropna()
    human = subset[human_col].tolist()
    llm = subset[llm_col].tolist()

    # Calculate percent agreement
    agreement = np.sum(np.array(human) == np.array(llm))/len(np.array(human)) * 100

    print("---")
    print(f"{classification_report(human, llm)}")
    print(f"cohen_kappa_score({human_col}): {cohen_kappa_score(human, llm)}")
    print(f"krippendorf({human_col}): {krippendorff.alpha(np.array([human, llm]), level_of_measurement="nominal")}")
    print(f"percent_agreement({human_col}): {agreement:.2f}%")


In [25]:
#evaluate performance of BEST prompts on the TEST set:
llm_human_column_pairs = [
    ("RATIONALITY_DUMMY", "rationality_simple2_dum"),
    ("INCIVILITY_DUMMY", "incivility_simple2_dum"),
    ("INTERACTIVITY_DUMMY", "interactivity_acknowledgement_simple_dum"),
    ("LIBERAL_DUMMY", "political_liberal_US_dum"),
    ("CONSERVATIVE_DUMMY", "political_conservative_US_dum")
]

print("Inter-coder reliablity: classification_report(human as gold, llm as predicted), cohen_kappa(human, llm):")

for human_col, llm_col in llm_human_column_pairs:
    subset = dataset_w_pred_anon.loc[dataset_w_pred_anon.split == 'test', [human_col, llm_col]].dropna()
    human = subset[human_col].tolist()
    llm = subset[llm_col].tolist()

    # Calculate percent agreement
    agreement = np.sum(np.array(human) == np.array(llm))/len(np.array(human)) * 100

    print("---")
    print(f"{classification_report(human, llm)}")
    print(f"cohen_kappa_score({human_col}): {cohen_kappa_score(human, llm)}")
    print(f"krippendorf({human_col}): {krippendorff.alpha(np.array([human, llm]), level_of_measurement="nominal")}")
    print(f"percent_agreement({human_col}): {agreement:.2f}%")


Inter-coder reliablity: classification_report(human as gold, llm as predicted), cohen_kappa(human, llm):
---
              precision    recall  f1-score   support

           0       0.85      0.94      0.89       624
           1       0.56      0.30      0.39       149

    accuracy                           0.82       773
   macro avg       0.70      0.62      0.64       773
weighted avg       0.79      0.82      0.80       773

cohen_kappa_score(RATIONALITY_DUMMY): 0.2912993935901299
krippendorf(RATIONALITY_DUMMY): 0.2802092484625829
percent_agreement(RATIONALITY_DUMMY): 81.89%
---
              precision    recall  f1-score   support

           0       0.86      0.64      0.74       408
           1       0.69      0.88      0.77       365

    accuracy                           0.76       773
   macro avg       0.77      0.76      0.75       773
weighted avg       0.78      0.76      0.75       773

cohen_kappa_score(INCIVILITY_DUMMY): 0.5165280641200862
krippendorf(INCIVILITY_D

In [8]:
def load_json(path: str):
    with open(path, encoding='utf-8') as fp:
        return json.load(fp)

In [9]:
#check GPT4o:
def load_json(path: str):
    with open(path, encoding='utf-8') as fp:
        return json.load(fp)
    

pubsphereazurerun4 = ['rationality_simple2']   
pubsphereazurerun5 = ['interactivity_acknowledgement_simple', 'incivility_simple2', 'political_ideology_US']
pubsphereazurerun6= ['incivility_simple2', 'political_ideology_US']   


chunked_result: typing.List[pd.DataFrame] = []

for label, path in CFG.prompt_classify_files.items():
    if label in pubsphereazurerun5: 
        template = load_json(path).get('template')
        classes = load_json(path).get('classes')
        for index, row in tqdm.tqdm(dataset["commentText"].items()):
            retry_count = 0
            max_retries = 5
            while retry_count < max_retries:
                try: 
                    response = requests.post(
                            url=api_endpoint,
                            headers=headers,
                            json={
                                'model': MODELgpt4o,
                                'messages': [
                                    {
                                        "role": "system",
                                        "content": template
                                    },
                                    {
                                        "role": "user",
                                        "content": template.format(text=row)
                                    }
                                ],
                                'temperature': temperature_0,  
                                'seed': SEED,
                                "max_tokens": MAX10
                            }
                        )  

                    if response.status_code == 200:
                        data_response = response.json()
                        chunked_result.append(
                        pd.DataFrame(
                            data=[[index, classes.get(data_response["choices"][0]["message"]["content"], None)]],                                
                            columns=['index', label]
                            )
                        )
                        break  # Exit the retry loop on success
                    elif response.status_code == 429:
                        retry_count += 1
                        wait_time = 20
                        print(f"Rate limit exceeded. Retrying in {wait_time} seconds...")
                        time.sleep(wait_time)
                    elif response.status_code == 500:
                        retry_count += 1
                        wait_time = 20
                        print(f"Failed to connect to API. Status code: {response.status_code}. Retrying in {wait_time} seconds...")
                        time.sleep(wait_time)
                    else:
                        print(f"Failed to connect to API. Status code: {response.status_code}")
                        print(response.text)
                        break
                except requests.exceptions.RequestException as e:   
                    print(f"Failed to connect to API: {e}")
                    retry_count += 1
                    wait_time = 60
                    print(f"Retrying in {wait_time} seconds...")
                    time.sleep(wait_time)                 
                
            #Add a pause of 1 minute after every 70 iterations
            #if (index + 1) % 250 == 0:
            #    print("Pausing for 10 seconds...")
            #    time.sleep(10)

classifications3d = pd.concat(chunked_result, ignore_index=True)
print(classifications3d) 

3862it [32:34,  1.98it/s]
3862it [33:07,  1.94it/s]
3862it [32:04,  2.01it/s]


       index incivility_simple2 interactivity_acknowledgement_simple  \
0          0                 No                                  NaN   
1          1                Yes                                  NaN   
2          2                 No                                  NaN   
3          3                 No                                  NaN   
4          4                 No                                  NaN   
...      ...                ...                                  ...   
11581   3857                NaN                                  NaN   
11582   3858                NaN                                  NaN   
11583   3859                NaN                                  NaN   
11584   3860                NaN                                  NaN   
11585   3861                NaN                                  NaN   

      political_ideology_US  
0                       NaN  
1                       NaN  
2                       NaN  
3              

In [10]:
classifications3c.set_index('index', drop=True, inplace=True)
dataset_w_pred_anon = dataset_w_pred_anon.join(classifications3c, rsuffix='_gpt4o_system')

In [10]:
incivility_gpt4o_system = classifications3d.loc[:, ["index", "incivility_simple2"]].dropna()
interactivity_gpt4o_system = classifications3d.loc[:, ["index", "interactivity_acknowledgement_simple"]].dropna()
political_ideology_US_gpt4o_system = classifications3d.loc[:, ["index", "political_ideology_US"]].dropna()

In [ ]:
incivility_gpt4o_system.set_index('index', drop=True, inplace=True)
interactivity_gpt4o_system.set_index('index', drop=True, inplace=True)
political_ideology_US_gpt4o_system.set_index('index', drop=True, inplace=True)
dataset_w_pred_anon = dataset_w_pred_anon.join(incivility_gpt4o_system, rsuffix='_gpt4o_system').join(interactivity_gpt4o_system, rsuffix='_gpt4o_system').join(political_ideology_US_gpt4o_system, rsuffix='_gpt4o_system')

In [17]:
dataset_w_pred_anon.loc[:, 'incivility_simple2_gpt4o_system_dum'] = dataset_w_pred_anon.loc[:, 'incivility_simple2_gpt4o_system'].map({"Yes": 1, "No":0}).fillna(0).astype(int)
dataset_w_pred_anon.loc[:, 'rationality_simple2_gpt4o_system_dum'] = dataset_w_pred_anon.loc[:, 'rationality_simple2_gpt4o_system'].map({"Yes": 1, "No":0}).fillna(0).astype(int)
dataset_w_pred_anon.loc[:, 'interactivity_acknowledgement_simple_gpt4o_system_dum'] = dataset_w_pred_anon.loc[:, 'interactivity_acknowledgement_simple_gpt4o_system'].map({"Yes": 1, "No":0}).fillna(0).astype(int)

dataset_w_pred_anon.loc[:, 'political_conservative_US_gpt4o_system_dum'] = dataset_w_pred_anon.loc[:, 'political_ideology_US_gpt4o_system'].map({"conservative": 1}).fillna(0).astype(int)
dataset_w_pred_anon.loc[:, 'political_liberal_US_gpt4o_system_dum'] = dataset_w_pred_anon.loc[:, 'political_ideology_US_gpt4o_system'].map({"liberal": 1}).fillna(0).astype(int)
dataset_w_pred_anon.loc[:, 'political_opinion_US_gpt4o_system_dum'] = dataset_w_pred_anon.loc[:, 'political_ideology_US_gpt4o_system'].map({"liberal": 1, 'conservative': 1}).fillna(0).astype(int)

In [21]:
#check the effect on the training set:
llm_human_column_pairs = [
    ("RATIONALITY_DUMMY", "rationality_simple2_gpt4o_system_dum"),
    ("INCIVILITY_DUMMY", "incivility_simple2_gpt4o_system_dum"),
    ("INTERACTIVITY_DUMMY", "interactivity_acknowledgement_simple_gpt4o_system_dum"),
    ("LIBERAL_DUMMY", "political_liberal_US_gpt4o_system_dum"),
    ("CONSERVATIVE_DUMMY", "political_conservative_US_gpt4o_system_dum")  
]

print("Inter-coder reliablity: classification_report(human as gold, llm as predicted), cohen_kappa(human, llm):")

for human_col, llm_col in llm_human_column_pairs:
    subset = dataset_w_pred_anon.loc[dataset_w_pred_anon.split == 'train', [human_col, llm_col]].dropna()
    human = subset[human_col].tolist()
    llm = subset[llm_col].tolist()

    # Calculate percent agreement
    agreement = np.sum(np.array(human) == np.array(llm))/len(np.array(human)) * 100

    print("---")
    print(f"{classification_report(human, llm)}")
    print(f"cohen_kappa_score({human_col}): {cohen_kappa_score(human, llm)}")
    print(f"krippendorf({human_col}): {krippendorff.alpha(np.array([human, llm]), level_of_measurement="nominal")}")
    print(f"percent_agreement({human_col}): {agreement:.2f}%")

Inter-coder reliablity: classification_report(human as gold, llm as predicted), cohen_kappa(human, llm):
---
              precision    recall  f1-score   support

           0       0.85      0.99      0.91      2541
           1       0.80      0.19      0.30       548

    accuracy                           0.85      3089
   macro avg       0.82      0.59      0.61      3089
weighted avg       0.84      0.85      0.81      3089

cohen_kappa_score(RATIONALITY_DUMMY): 0.2514882877554834
krippendorf(RATIONALITY_DUMMY): 0.21611506520490675
percent_agreement(RATIONALITY_DUMMY): 84.72%
---
              precision    recall  f1-score   support

           0       0.68      0.90      0.78      1567
           1       0.85      0.57      0.68      1522

    accuracy                           0.74      3089
   macro avg       0.77      0.74      0.73      3089
weighted avg       0.77      0.74      0.73      3089

cohen_kappa_score(INCIVILITY_DUMMY): 0.47486824956083185
krippendorf(INCIVILITY

In [33]:
#check effect on GPT4-turbo:
def load_json(path: str):
    with open(path, encoding='utf-8') as fp:
        return json.load(fp)
    

pubsphereazurerun6 = ['rationality_simple2']
pubsphereazurerun7 = ['interactivity_acknowledgement_simple']
pubsphereazurerun8 = ['incivility_simple2']
pubsphereazurerun9 = ['political_ideology_US']


chunked_result: typing.List[pd.DataFrame] = []

for label, path in CFG.prompt_classify_files.items():
    if label in pubsphereazurerun9: 
        template = load_json(path).get('template')
        classes = load_json(path).get('classes')
        for index, row in tqdm.tqdm(dataset["commentText"].items()):
            retry_count = 0
            max_retries = 5
            while retry_count < max_retries:
                try: 
                    response = requests.post(
                            url=api_endpoint,
                            headers=headers,
                            json={
                                'model': MODELgpt4T,
                                'messages': [
                                    {
                                        "role": "system",
                                        "content": template
                                    },
                                    {
                                        "role": "user",
                                        "content": template.format(text=row)
                                    }
                                ],
                                'temperature': temperature_0,  
                                'seed': SEED,
                                "max_tokens": MAX10
                            }
                        )  

                    if response.status_code == 200:
                        data_response = response.json()
                        chunked_result.append(
                        pd.DataFrame(
                            data=[[index, classes.get(data_response["choices"][0]["message"]["content"], None)]],                                
                            columns=['index', label]
                            )
                        )
                        break  # Exit the retry loop on success
                    elif response.status_code == 429:
                        retry_count += 1
                        wait_time = 20
                        print(f"Rate limit exceeded. Retrying in {wait_time} seconds...")
                        time.sleep(wait_time)
                    elif response.status_code == 500:
                        retry_count += 1
                        wait_time = 20
                        print(f"Failed to connect to API. Status code: {response.status_code}. Retrying in {wait_time} seconds...")
                        time.sleep(wait_time)
                    else:
                        print(f"Failed to connect to API. Status code: {response.status_code}")
                        print(response.text)
                        break
                except requests.exceptions.RequestException as e:   
                    print(f"Failed to connect to API: {e}")
                    retry_count += 1
                    wait_time = 60
                    print(f"Retrying in {wait_time} seconds...")
                    time.sleep(wait_time)                 
                
            #Add a pause of 1 minute after every 70 iterations
            #if (index + 1) % 250 == 0:
            #    print("Pausing for 10 seconds...")
            #    time.sleep(10)

classifications9 = pd.concat(chunked_result, ignore_index=True)
print(classifications9) 

15it [00:06,  2.23it/s]

Rate limit exceeded. Retrying in 20 seconds...


30it [00:36,  1.88it/s]

Rate limit exceeded. Retrying in 20 seconds...
Rate limit exceeded. Retrying in 20 seconds...


45it [01:27,  1.88it/s]

...

Rate limit exceeded. Retrying in 20 seconds...
Rate limit exceeded. Retrying in 20 seconds...


3852it [2:43:15,  1.62it/s]

Rate limit exceeded. Retrying in 20 seconds...
Rate limit exceeded. Retrying in 20 seconds...


3862it [2:44:03,  2.55s/it]

      index political_ideology_US
0         0               neutral
1         1               neutral
2         2               liberal
3         3               neutral
4         4               neutral
...     ...                   ...
3857   3857               liberal
3858   3858               neutral
3859   3859               neutral
3860   3860               neutral
3861   3861               neutral

[3862 rows x 2 columns]


In [ ]:
#check model version:
data_response["model"]

'gpt-4-turbo-2024-04-09'

In [ ]:
#budget spend by classifying pubsphere on GPT4T and GPT4o once: 26.87183725000002

In [20]:
rationality_gpt4T_system = classifications4append.loc[:, ["index", "rationality_simple2"]].dropna()
rationality_gpt4T_system.set_index('index', drop=True, inplace=True)
dataset_w_pred_anon = dataset_w_pred_anon.join(rationality_gpt4T_system, rsuffix='_gpt4T_system')
dataset_w_pred_anon.loc[:, 'rationality_simple2_gpt4T_system_dum'] = dataset_w_pred_anon.loc[:, 'rationality_simple2_gpt4T_system'].map({"Yes": 1, "No":0}).fillna(0).astype(int)


In [25]:
interactivity_gpt4T_system = classifications7.loc[:, ["index", "interactivity_acknowledgement_simple"]].dropna()
interactivity_gpt4T_system.set_index('index', drop=True, inplace=True)
dataset_w_pred_anon = dataset_w_pred_anon.join(interactivity_gpt4T_system, rsuffix='_gpt4T_system')
dataset_w_pred_anon.loc[:, 'interactivity_acknowledgement_simple_gpt4T_system_dum'] = dataset_w_pred_anon.loc[:, 'interactivity_acknowledgement_simple_gpt4T_system'].map({"Yes": 1, "No":0}).fillna(0).astype(int)



In [30]:
incivility_gpt4T_system = classifications8.loc[:, ["index", "incivility_simple2"]].dropna()
incivility_gpt4T_system.set_index('index', drop=True, inplace=True)
dataset_w_pred_anon = dataset_w_pred_anon.join(incivility_gpt4T_system, rsuffix='_gpt4T_system')
dataset_w_pred_anon.loc[:, 'incivility_simple2_gpt4T_system_dum'] = dataset_w_pred_anon.loc[:, 'incivility_simple2_gpt4T_system'].map({"Yes": 1, "No":0}).fillna(0).astype(int)


In [37]:
political_ideology_US_gpt4T_system = classifications9.loc[:, ["index", "political_ideology_US"]].dropna()
political_ideology_US_gpt4T_system.set_index('index', drop=True, inplace=True)
dataset_w_pred_anon = dataset_w_pred_anon.join(political_ideology_US_gpt4T_system, rsuffix='_gpt4T_system')
dataset_w_pred_anon.loc[:, 'political_conservative_US_gpt4T_system_dum'] = dataset_w_pred_anon.loc[:, 'political_ideology_US_gpt4T_system'].map({"conservative": 1}).fillna(0).astype(int)
dataset_w_pred_anon.loc[:, 'political_liberal_US_gpt4T_system_dum'] = dataset_w_pred_anon.loc[:, 'political_ideology_US_gpt4T_system'].map({"liberal": 1}).fillna(0).astype(int)


In [ ]:
rationality_gpt4T_system = classifications4.loc[:, ["index", "rationality_simple2"]].dropna()
incivility_gpt4T_system = classifications4.loc[:, ["index", "incivility_simple2"]].dropna()
interactivity_gpt4T_system = classifications4.loc[:, ["index", "interactivity_acknowledgement_simple"]].dropna()
political_ideology_US_gpt4T_system = classifications4.loc[:, ["index", "political_ideology_US"]].dropna()

In [ ]:
rationality_gpt4T_system.set_index('index', drop=True, inplace=True)
incivility_gpt4T_system.set_index('index', drop=True, inplace=True)
interactivity_gpt4T_system.set_index('index', drop=True, inplace=True)
political_ideology_US_gpt4T_system.set_index('index', drop=True, inplace=True)
dataset_w_pred_anon = dataset_w_pred_anon.join(rationality_gpt4T_system, rsuffix='_gpt4T_system').join(incivility_gpt4T_system, rsuffix='_gpt4T_system').join(interactivity_gpt4T_system, rsuffix='_gpt4T_system').join(political_ideology_US_gpt4T_system, rsuffix='_gpt4T_system')

In [ ]:
dataset_w_pred_anon.loc[:, 'incivility_simple2_gpt4T_system_dum'] = dataset_w_pred_anon.loc[:, 'incivility_simple2_gpt4T_system'].map({"Yes": 1, "No":0}).fillna(0).astype(int)
dataset_w_pred_anon.loc[:, 'rationality_simple2_gpt4T_system_dum'] = dataset_w_pred_anon.loc[:, 'rationality_simple2_gpt4T_system'].map({"Yes": 1, "No":0}).fillna(0).astype(int)
dataset_w_pred_anon.loc[:, 'interactivity_acknowledgement_simple_gpt4T_system_dum'] = dataset_w_pred_anon.loc[:, 'interactivity_acknowledgement_simple_gpt4T_system'].map({"Yes": 1, "No":0}).fillna(0).astype(int)

dataset_w_pred_anon.loc[:, 'political_conservative_US_gpt4T_system_dum'] = dataset_w_pred_anon.loc[:, 'political_ideology_US_gpt4T_system'].map({"conservative": 1}).fillna(0).astype(int)
dataset_w_pred_anon.loc[:, 'political_liberal_US_gpt4T_system_dum'] = dataset_w_pred_anon.loc[:, 'political_ideology_US_gpt4T_system'].map({"liberal": 1}).fillna(0).astype(int)
dataset_w_pred_anon.loc[:, 'political_opinion_US_gpt4T_system_dum'] = dataset_w_pred_anon.loc[:, 'political_ideology_US_gpt4T_system'].map({"liberal": 1, 'conservative': 1}).fillna(0).astype(int)

In [4]:
#save data:
dataset_w_pred_anon.to_json('data/publicsphere/publicsphere.cardiff_prompt_classify_anon.json', orient="records", force_ascii=False, indent=4)
dataset_w_pred_anon.to_parquet('data/publicsphere/publicsphere.cardiff_prompt_classify_anon.parquet')

In [5]:
dataset_w_pred_anon

,StartDate,RecordedDate,IPAddress,Finished,Coder,ID,Mark_ID,Genre,topiccode,Platform,...,political_opinion_US_gpt4o_system_dum,rationality_simple2_gpt4T_system,rationality_simple2_gpt4T_system_dum,interactivity_acknowledgement_simple_gpt4T_system,interactivity_acknowledgement_simple_gpt4T_system_dum,incivility_simple2_gpt4T_system,incivility_simple2_gpt4T_system_dum,political_ideology_US_gpt4T_system,political_conservative_US_gpt4T_system_dum,political_liberal_US_gpt4T_system_dum
0,5/30/2021 13:03:17,5/30/2021 13:04:17,62.194.51.29,1,6,UgyPHwv8G0cDE6-wEgl4AaABAg.8_0ZjJKSJty8_0kXGkAd2U,119,0,0,1,...,0,No,0,No,0,No,0,neutral,0,0
1,10/11/2021 10:34:05,10/11/2021 10:36:46,213.127.109.191,1,6,Ugx2WXq9UdV8mPPjejJ4AaABAg.8yHCKV0Boe58yYRxEQEF45,282,1,2,1,...,0,No,0,No,0,Yes,1,neutral,0,0
2,9/9/2021 18:49:48,9/9/2021 18:51:32,213.127.110.0,1,6,1110578710648890000,372,2,4,2,...,1,No,0,Yes,1,No,0,liberal,0,1
3,6/6/2021 16:12:46,6/6/2021 16:16:16,213.127.76.145,1,6,UgwUPFScjJ0MCeaP2F54AaABAg.8lvp3fc9Euf8lvvgsUgEgV,769,0,0,1,...,0,No,0,No,0,No,0,neutral,0,0
4,6/13/2021 13:25:49,6/13/2021 13:27:28,213.127.82.232,1,6,UgwWKCWtSJdFvjGHvTp4AaABAg.8kUC5dGrQ2H8kUDRihE2f3,1206,0,0,1,...,0,No,0,No,0,No,0,neutral,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3857,8/19/2021 14:50:13,8/19/2021 14:54:28,62.194.51.29,1,6,1152219467579100000,10000695,0,4,2,...,0,No,0,Yes,1,Yes,1,liberal,0,1
3858,8/19/2021 15:10:27,8/19/2021 15:12:21,62.194.51.29,1,6,1085362296472430000,10007008,1,4,2,...,0,No,0,No,0,No,0,neutral,0,0
3859,10/6/2021 16:08:39,10/6/2021 16:10:42,213.127.113.113,1,6,UghFY3QJ6nmT_ngCoAEC.7-H0Z7--wxd8goqpaPs-bl,20000102,0,3,1,...,0,No,0,No,0,No,0,neutral,0,0
3860,10/15/2021 18:30:04,10/15/2021 18:35:40,213.127.109.191,1,6,UgyWabsmmnq3zam4DgZ4AaABAg,20000418,2,3,1,...,0,No,0,No,0,No,0,neutral,0,0


In [3]:
# check the effect on the training set:
llm_human_column_pairs = [
    ("RATIONALITY_DUMMY", "rationality_simple2_gpt4T_system_dum"),
    ("INCIVILITY_DUMMY", "incivility_simple2_gpt4T_system_dum"),
    ("INTERACTIVITY_DUMMY", "interactivity_acknowledgement_simple_gpt4T_system_dum"),
    ("LIBERAL_DUMMY", "political_liberal_US_gpt4T_system_dum"),
    ("CONSERVATIVE_DUMMY", "political_conservative_US_gpt4T_system_dum")  
]

print("Inter-coder reliablity: classification_report(human as gold, llm as predicted), cohen_kappa(human, llm):")

for human_col, llm_col in llm_human_column_pairs:
    subset = dataset_w_pred_anon.loc[dataset_w_pred_anon.split == 'train', [human_col, llm_col]].dropna()
    human = subset[human_col].tolist()
    llm = subset[llm_col].tolist()

    # Calculate percent agreement
    agreement = np.sum(np.array(human) == np.array(llm))/len(np.array(human)) * 100

    print("---")
    print(f"{classification_report(human, llm)}")
    print(f"cohen_kappa_score({human_col}): {cohen_kappa_score(human, llm)}")
    print(f"krippendorf({human_col}): {krippendorff.alpha(np.array([human, llm]), level_of_measurement="nominal")}")
    print(f"percent_agreement({human_col}): {agreement:.2f}%")

Inter-coder reliablity: classification_report(human as gold, llm as predicted), cohen_kappa(human, llm):
---
              precision    recall  f1-score   support

           0       0.84      0.99      0.91      2541
           1       0.84      0.14      0.24       548

    accuracy                           0.84      3089
   macro avg       0.84      0.57      0.57      3089
weighted avg       0.84      0.84      0.79      3089

cohen_kappa_score(RATIONALITY_DUMMY): 0.1973134272611098
krippendorf(RATIONALITY_DUMMY): 0.15008726003490402
percent_agreement(RATIONALITY_DUMMY): 84.23%
---
              precision    recall  f1-score   support

           0       0.74      0.87      0.80      1567
           1       0.83      0.69      0.75      1522

    accuracy                           0.78      3089
   macro avg       0.79      0.78      0.78      3089
weighted avg       0.79      0.78      0.78      3089

cohen_kappa_score(INCIVILITY_DUMMY): 0.5539779563940896
krippendorf(INCIVILITY_